# 环境

In [1]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

# 加载unsloth框架

In [1]:
model_path = "/home/yxlin/huggingface/LLaMA-2-7b-bnb-4bit"

from unsloth import FastLanguageModel
import torch

max_seq_length = 2048   # 官方教程推荐
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_path,    ### Do not change the model for any other models or quantization versions
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/yxlin/github/LHY_ML2025/.venv/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.3: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 2. Max memory: 23.516 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


## 微调之前 对话

In [2]:
# 1. 切换到推理模式（Unsloth 专用加速）
model = FastLanguageModel.for_inference(model) 

# 2. 定义你的原生输入（直接写你想说的话，不加任何包装）
text = "你好, 今天天气不错。所以今天星期几？"

# 3. 编码并生成
inputs = tokenizer([text], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 512)

# 4. 解码输出
response = tokenizer.batch_decode(outputs)
print(response[0])

<s> 你好, 今天天气不错。所以今天星期几？
The sun is shining brightly today. It is a beautiful day.
今天是一个漂亮的日子，天冷凉，太阳闪烁着，吹来了一阵清涼的东北风，好了，今天星期几？
This is a beautiful day.
今天是一个漂亮的日子。
The sun is shining brightly today.
天冷凉，太阳闪烁着。
It is a beautiful day.
好了，今天星期几？
Today is a beautiful day.
今天是一个漂亮的日子。天冷凉，太阳闪烁着。
It is a beautiful day. Today is a beautiful day.
今天星期几？
What is the weather like today?
今天天气怎么样？
It is a beautiful day. Today is a beautiful day.
天冷凉，太阳闪烁着，吹来了一阵清涼的东北风，好了，今天星期几？
It is a beautiful day. Today is a beautiful day. 天冷凉，太阳闪烁着，吹来了一阵清涼的东北风，好了，今天星期几？
It is a beautiful day. Today is a beautiful day. 天冷凉，太阳闪烁着，吹来了一阵清涼的东北风，好了，今天星期几？
天冷凉，太阳闪烁着，吹来了一阵清涼的东北风，好了，今天星期几？ 天冷凉，太阳闪烁着，


In [3]:
print(tokenizer.chat_template)

None


## 加载微调框架代码

In [4]:
################# TODO : Tweak the LoRA adapter hyperparameters here.  #####################

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, ### TODO : Choose any number > 0 ! Common values are 4, 8, 16, 32, 64, 128. Higher ranks allow more expressive power but also increase parameter count.
    lora_alpha = 16,  ### TODO : Choose any number > 0 ! Suggested 4, 8, 16, 32, 64, 128


################# TODO  ####################################################################
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2026.1.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# 获取数据

* 数据位于[tatsu-lab/alpaca](https://huggingface.co/datasets/tatsu-lab/alpaca)

In [5]:
from datasets import load_dataset

data_path = "/home/yxlin/github/LHY_ML2025/MLHW5/dataset/data/"
data_name = "train-00000-of-00001-a09b74b3ef9c3b56.parquet"

dataset = load_dataset(
    data_path,
    data_files=data_name
)

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 52002
    })
})


我需要选择出100条数据进行微调

In [6]:
from datasets import load_dataset, Dataset, load_from_disk

def compute_conversation_length(message):
    # 增加一个简单的安全检查，防止 input 为 None
    instruction = message['instruction'] or ""
    input_text = message['input'] or ""
    output = message['output'] or ""
    return len(instruction) + len(input_text) + len(output)

# 修改这里：使用 dataset['train']
sorted_dataset_list = sorted(dataset['train'], key=compute_conversation_length, reverse=False)

# 注意：sorted_dataset_list 现在是一个 Python List（列表），不再是 HuggingFace Dataset 对象
print(sorted_dataset_list[0])

sorted_dataset = Dataset.from_list(sorted_dataset_list)
print("\nTop examples sorted by simple conversation length:")
for id, entry in enumerate(sorted_dataset.select(range(5))):
    print(f"ID: {id}, Conversation Length: {compute_conversation_length(entry)}")

train_dataset = sorted_dataset.select(range(0,100))

{'instruction': 'Solve 8 x 8.', 'input': '', 'output': '64', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nSolve 8 x 8.\n\n### Response:\n64'}

Top examples sorted by simple conversation length:
ID: 0, Conversation Length: 14
ID: 1, Conversation Length: 14
ID: 2, Conversation Length: 15
ID: 3, Conversation Length: 15
ID: 4, Conversation Length: 15


## 多列用于微调 与 模板

In [7]:
from unsloth.chat_templates import get_chat_template

# # Llama-2 官方格式
# llama2_official_template = """<s>[INST] <<SYS>>
# {SYSTEM}
# <</SYS>>

# {INPUT} [/INST] {OUTPUT} </s>"""

# tokenizer = get_chat_template(
#     tokenizer,
#     chat_template = (llama2_official_template, "</s>"), # 只保留这两个，删掉 True 和 None
# )

# system_prompt = "You are a AI Helper. Below is an instruction that describes a task. Write a response that appropriately completes the request."

# def formatting_prompts_func(examples):
#     instructions = examples["instruction"]
#     inputs       = examples["input"]
#     outputs      = examples["output"]
#     texts = []
    
#     for instruction, input, output in zip(instructions, inputs, outputs):
#         # 1. 按照你之前的逻辑合并指令和输入
#         combined_input = f"{instruction}\n{input}" if input else instruction
        
#         # 2. 直接使用 Python 的 .format() 填充模板字符串
#         # 这里的变量名必须和你模板里的 {SYSTEM}, {INPUT}, {OUTPUT} 一一对应
#         text = llama2_official_template.format(
#             SYSTEM = system_prompt,
#             INPUT  = combined_input,
#             OUTPUT = output
#         )
#         texts.append(text)
        
#     return { "text" : texts, }

# train_dataset = train_dataset.map(formatting_prompts_func, batched = True)

bos_token = "<s>"
eos_token = "</s>"

custom_template = """{bos_token}<|im_start|>system
{SYSTEM}<|im_end|>
<|im_start|>user
{INPUT}<|im_end|>
<|im_start|>assistant
{OUTPUT}<|im_end|>{eos_token}"""

tokenizer = get_chat_template(
    tokenizer,
    chat_template = (custom_template, eos_token),
)

system_prompt = "You are a AI Helper. Below is an instruction that describes a task. Write a response that appropriately completes the request."

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    
    for instruction, input, output in zip(instructions, inputs, outputs):
        # 1. 按照你之前的逻辑合并指令和输入
        combined_input = f"{instruction}\n{input}" if input else instruction
        
        # 2. 直接使用 Python 的 .format() 填充模板字符串
        # 这里的变量名必须和你模板里的 {SYSTEM}, {INPUT}, {OUTPUT} 一一对应
        text = custom_template.format(
            SYSTEM = system_prompt,
            INPUT  = combined_input,
            OUTPUT = output,
            bos_token = bos_token,
            eos_token = eos_token,
        )
        texts.append(text)
        
    return { "text" : texts, }

train_dataset = train_dataset.map(formatting_prompts_func, batched = True)

Map: 100%|██████████| 100/100 [00:00<00:00, 4393.51 examples/s]


In [8]:
print(tokenizer.chat_template)

{bos_token}<|im_start|>system
{SYSTEM}<|im_end|>
<|im_start|>user
{INPUT}<|im_end|>
<|im_start|>assistant
{OUTPUT}<|im_end|>{eos_token}


In [127]:
import random

for i in range(3):
    idx = random.randint(0, len(train_dataset) - 1)
    print(f"--- 样本 {idx} ---")
    print(train_dataset[idx]["text"])
    print("\n" + "="*50 + "\n")

--- 样本 7 ---
<s><|im_start|>system
You are a AI Helper. Below is an instruction that describes a task. Write a response that appropriately completes the request.<|im_end|>
<|im_start|>user
Multiply 4 and 7.<|im_end|>
<|im_start|>assistant
28<|im_end|></s>


--- 样本 53 ---
<s><|im_start|>system
You are a AI Helper. Below is an instruction that describes a task. Write a response that appropriately completes the request.<|im_end|>
<|im_start|>user
Calculate 17 + 18.<|im_end|>
<|im_start|>assistant
17 + 18 = 35<|im_end|></s>


--- 样本 16 ---
<s><|im_start|>system
You are a AI Helper. Below is an instruction that describes a task. Write a response that appropriately completes the request.<|im_end|>
<|im_start|>user
Calculate 34 X 65.<|im_end|>
<|im_start|>assistant
2210<|im_end|></s>




# 训练

In [128]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported


################# TODO : Tweak the training hyperparameters here.  #####################


training_config = {
    "per_device_train_batch_size": 2,
    "gradient_accumulation_steps": 4,
    "warmup_steps": 10,
    "num_train_epochs": 2,
    "learning_rate": 2e-4,
    "optim": "adamw_8bit",
    "weight_decay": 0.01,
    "lr_scheduler_type": "linear",
    "seed": 3407,   ### Do not modify the seed for reproducibility
}


################# TODO #################################################################

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = training_config["per_device_train_batch_size"],
        gradient_accumulation_steps = training_config["gradient_accumulation_steps"],
        warmup_steps = training_config["warmup_steps"],
        num_train_epochs = training_config["num_train_epochs"], # Set this for 1 full training run.
        # max_steps = 60,
        learning_rate = training_config["learning_rate"],
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = training_config["optim"],
        weight_decay = training_config["weight_decay"],
        lr_scheduler_type = training_config["lr_scheduler_type"],
        seed = training_config["seed"],
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=84): 100%|██████████| 100/100 [00:07<00:00, 12.75 examples/s]


In [129]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100 | Num Epochs = 2 | Total steps = 14
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 39,976,960 of 6,778,392,576 (0.59% trained)


Step,Training Loss
1,2.561300
2,2.518300
3,2.537100
4,2.523100
5,2.490400
6,2.408200
7,2.238800
8,2.031400
9,1.708700
10,1.382800


# Inference

In [130]:
# 1. 切换到推理模式（Unsloth 专用加速）
model = FastLanguageModel.for_inference(model)

In [ ]:
system_prompt = "You are a AI Helper. Below is an instruction that describes a task. Write a response that appropriately completes the request."
input_text = "你好, 今天天气不错。所以今天星期几？" 

# 2. 定义你的原生输入
# text = """<s>[INST] <<SYS>>
# {SYSTEM}
# <</SYS>>

# {INPUT} [/INST]""".format(
#         SYSTEM = system_prompt,
#         INPUT  = input_text,
# )
# text = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
# {SYSTEM}<|eot_id|><|start_header_id|>user<|end_header_id|>
# {INPUT}<|eot_id|><|start_header_id|>assistant<|end_header_id|>""".format(
#         SYSTEM = system_prompt,
#         INPUT  = input_text,
# )
text = """<s><|im_start|>system
{SYSTEM}<|im_end|>
<|im_start|>user
{INPUT}<|im_end|>
<|im_start|>assistant""".format(
         SYSTEM = system_prompt,
         INPUT  = input_text,
)

# 3. 编码并生成
inputs = tokenizer([text], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 512, eos_token_id = tokenizer.eos_token_id,)

# 4. 解码输出
response = tokenizer.batch_decode(outputs)
print(response[0])

<s><s> <|im_start|>system
You are a AI Helper. Below is an instruction that describes a task. Write a response that appropriately completes the request.<|im_end|>
<|im_start|>user
你好, 今天天气不错。所以今天星期几？<|im_end|>
<|im_start|>assistant
星期天<|im_end|></s>


: 

上述可以看到在微调前和微调后回答有巨大差异

我使用了两套模板，对比发现明显是如下这个模板更好：
```
# llama2_official_template = """<s>[INST] <<SYS>>
# {SYSTEM}
# <</SYS>>

# {INPUT} [/INST] {OUTPUT} </s>"""
```

# 保存模型

In [53]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/chat_template.jinja',
 'lora_model/tokenizer.model',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')